## This is the main program file for the HHC-Tools GUI

In [1]:
#import background python modules#
import sys
#import numpy as np
import tkinter as tk
from tkinter import ttk, messagebox, StringVar, filedialog
#import matplotlib.pyplot as plt
from PIL import Image, ImageTk # pip install_pillow

In [2]:

#-------------------------#
m=tk.Tk()                            # Create instance for master window     
m.title("HHC Tools")                 # Add a title for master window
m.geometry("525x650")
tabControl = ttk.Notebook(m)      # Create Tab Control using ttk notebook

# Specify font size for tabs
s = ttk.Style()
s.configure('TNotebook.Tab', font=('Arial',14))
######################## Home Tab ############################
hometab = ttk.Frame(tabControl)            # Create overtopping tab instance 

# Add image to GUI
imge1 = Image.open(".\MS_birdfoot.jpg")
photo1 = ImageTk.PhotoImage(imge1)
lab_home = tk.Label(hometab,image=photo1)
lab_home.pack()

tabControl.add(hometab, text='Home')      # Add the tab
tabControl.pack(expand=1, fill="both")  # Pack to make visible

# Add title to top of GUI
title = tk.Label(hometab,text="HHC Tools",relief="solid",width=25,font=("arial",19,"bold"),bg='white')
title.place(x=75,y=50)

######################## Overtopping Tool Begin ############################

sys.path.insert(1,'.\overtopping\eurotop')

import EurOtop_Overtopping as ot
# from App_Overtopping import m as ot
overtopping = ttk.Frame(tabControl)            # Create overtopping tab instance 

# Add image to page
imge = Image.open(".\overtopping\eurotop\overtopping-manual-eurotop-image-01.jpg")
photo = ImageTk.PhotoImage(imge)
lab = tk.Label(overtopping,image=photo)
lab.pack()

# Define type of input
var_method = StringVar()
var_sim = StringVar()
var_in = StringVar()
var_out = StringVar()
def select_in():
    global ot_file_in
    ot_file_in = var_in.get()
    ot_file_in = filedialog.askopenfilename(initialdir = "/",
                                     title = "Select file",
                                     filetypes = (("XLS files","*.xls"),("CSV files","*.csv"),("all files","*.*")))
    return ot_file_in

    #if ot_file_in:
    #    print(f"Input file is {ot_file_in}")
    print(f"Input file is {ot_file_in}")
    #else:
    #    warnings.warn('Please select an input file')
def select_out():
    global ot_calc_method
    ot_calc_method = var_method.get() # calculation method
    global ot_file_out
    if ot_calc_method == 'Mean Value':
        ot_file_out =  filedialog.asksaveasfilename(initialdir = "/", 
                                                 initialfile = "100YR_Future2_Eurotop_MeanValue",
                                                 title = "Select file",filetypes = (("CSV files","*.csv"),("all files","*.*")))
        print(f"Output file is {ot_file_out}")
    elif ot_calc_method == 'Design & Assessment':
        ot_file_out =  filedialog.asksaveasfilename(initialdir = "/", 
                                                 initialfile = "100YR_Future2_Eurotop_DesignAssess", defaultextension = ".csv",title = "Select file",filetypes = (("CSV files","*.csv"),("all files","*.*")))
        print(f"Output file is {ot_file_out}")
    else:
        tk.messagebox.showinfo("Select output file",
                                    "Please select a calculation method.")
        #warnings.warn('Please select a calculation method')#ot_file_out =  filedialog.asksaveasfilename(initialdir = "/", initialfile = "100YR_Future2_Eurotop_DesignAssess", defaultextension = ".csv",title = "Select file",filetypes = (("CSV files","*.csv"),("all files","*.*")))
    return ot_file_out, ot_calc_method
    
def select_run(): # Options for run simulations button
    ot_calc_method = var_method.get() # calculation method
    global ot_numsim
    ot_numsim = var_sim.get() # number of iterations
        
    try: ot_file_in
    except NameError: tk.messagebox.showinfo("Run Calculation","Please select an input file.")#ot_file_in = '' # None
    try: ot_file_out
    except NameError: tk.messagebox.showinfo("Run Calculation","Please select an output file.")#ot_file_out = '' # None
        
    if (ot_calc_method == "Mean Value" or ot_calc_method == "Design & Assessment") and (ot_file_in != "" and ot_file_in != None) and (ot_file_out != "" and ot_file_out != None):
#     if (ot_calc_method == "Mean Value" or ot_calc_method == "Design & Assessment") and ot_file_in and ot_file_out:
        print(f"Method chosen: {ot_calc_method}")
        print(f"Number of iterations to perform: {ot_numsim}")
        tk.messagebox.showinfo("Overtopping Calculation Progress",
                                    "Program is running!")
#         m.destroy() # closes window
        # use the overtopping function
        ot.OT(ot_numsim,ot_calc_method,ot_file_in,ot_file_out)
        
    elif ot_calc_method != 'Mean Value' and  ot_calc_method !='Design & Assessment':
        tk.messagebox.showinfo("Run Calculation",
                                    "Please select a calculation method.")
    return ot_numsim


# Create drop down menu for calculation method (mean value vs. design/assess)
# Label for dropdown menu
label_1 = tk.Label(overtopping,text="Calculation Method:",width=20,font=("arial",12,"bold"),bg='white')
label_1.place(x=50,y=374)
# Dropdown menu
list1 = ['Mean Value', 'Design & Assessment']
droplist = tk.OptionMenu(overtopping,var_method,*list1)
var_method.set("Select Method")
droplist.config(width=17,font=("arial",12))
droplist.place(x=270,y=370)

# Enter text for number of iterations
label_2 = tk.Label(overtopping,text="Number of Iterations:",width=20,font=("arial",12,"bold"),bg='white')
label_2.place(x=50,y=410)
entry_2 = tk.Entry(overtopping,textvar=var_sim,width=22,font=("arial",12))
entry_2.place(x=268, y=411)
entry_2.insert(tk.END,20000)

# Create button to choose input file
#b_in = Button(m,text="Input File",width=12,bg=((16,24,31)),fg=(211,188,141) )#  ,command=select_in)
b_in = tk.Button(overtopping,text="Select Input File",width=19,bg="#101820",fg="#D3BC8D",font=("arial",12,"bold"),command=select_in)
b_in.place(x=50,y=500)

# Create button to choose where to save file
b_out = tk.Button(overtopping,text="Select Output Name",width=19,bg="#101820",fg="#D3BC8D",font=("arial",12,"bold"),command=select_out)
b_out.place(x=269,y=500)


# # Add progress bar
# prog_it = Progressbar(m, orient = HORIZONTAL, length = 100, mode = 'determinate') # progress bar for iteration number 
# prog_it.pack() # iteration number progress bar


# Create button to run simulations
b_run = tk.Button(overtopping,text="Run Simulations",width=15,bg="#101820",fg="#D3BC8D",font=("arial",14,"bold"),command=select_run)
b_run.place(x=175,y=550)

# Add title to top of page
title = tk.Label(overtopping,text="EurOtop Overtopping (2018)",relief="solid",width=25,font=("arial",19,"bold"),bg='white')
title.place(x=75,y=50)


tabControl.add(overtopping, text='Overtopping')      # Add the title to the tab 
tabControl.pack(expand=1, fill="both")   # Pack to make visible  
# button_ot = tk.Button(overtopping, text='Overtopping', width=25, command=open_ot)#m.destroy) # adding button to overtopping tab
# button_ot.pack()

#################### Overtopping Tool end ############################

################# Rainfall Tool Begin ##########################

sys.path.insert(2,'.\precipitation\predicted')
import rainfall_pred1 as prcp
rainfall = ttk.Frame(tabControl)            # Create rainfall tab instance 
tabControl.add(rainfall, text='Rainfall')      # Add the tab
tabControl.pack(expand=1, fill="both")  # Pack to make visible
button2 = tk.Button(rainfall, text='Rainfall', width=25, command=lambda : prcp.run_precip(30,-90,'all','24-hr')) # adding button to rainfall tab
button2.pack()

#Rainfall Tool end#

m.mainloop() # Start GUI


<Figure size 640x480 with 1 Axes>